<h1>Обработка данных и создание базы знаний (Часть 2)</h1>

In [2]:
import pandas as pd
import os
os.chdir(os.path.join('..', 'data', 'raw'))

In [3]:
contexts = pd.read_csv('contexts.csv', index_col=0)
contexts.head()

,context,id
0,В протерозойских отложениях органические остат...,1
1,Кишечник млекопитающего подразделяется на тонк...,2
2,Город Байконур и космодром Байконур вместе обр...,3
3,Вскоре после прибытия Колумба из Вест-Индии во...,4
4,Около Порт-Артура ночью на 27 января 1904 года...,5


<h3>Обработка текста</h3>

Для формирования базы знаний и реализации поиска предлагаю реализовать подход document retrieval на основе whoosh + sql. Whoosh был выбран для оптимизации поиска, поскольку хранит инвертированный индекс и может осуществлять поиск по алогоритмам TF-IDF и BM25, чтобы позволит достичь хорошего баланса между точностью и ресурсозатратами. В индексе whoosh хранится id документа:

*Schema(id=NUMERIC(stored=True), content=TEXT(stored=False))*

В базе данных хранятся оригиналы текстов, доступ к которым осуществляется по id.

Для оптимизации поиска необходимо обработать контексты и вопросы. Были выбраны следующие техники:
* приведение к нижнему регистру
* удаление пунктуации
* удаление стоп-слов
* исправление ошибок
* извлечение определенных частей речи
* лемматизация или стемминг
* удаление часто встречающихся слов

In [6]:
import re
import nltk
from pyaspeller import YandexSpeller
from pymystem3 import Mystem
from nltk.corpus import stopwords
nltk.download('stopwords', quiet=True)

True

Я не нашла качественных стеммеров для русскоязычных текстов. Поэтому вместо SnowballStemmer было решено использовать pymystem3 от Яндекса.

In [93]:
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize

stemmer = SnowballStemmer("russian")
text = "Погода играет большую, а иногда даже решающую роль в человеческой истории. Помимо изменений климата, которые вызывали постепенную миграцию народов (например, опустынивание Ближнего Востока и формирование сухопутных мостов между материками во время ледниковых периодов), экстремальные погодные явления вызывали меньшие по масштабу перемещения народов и принимали непосредственное участие в исторических событиях. Одним из таких случаев является спасение Японии ветрами Камикадзе от вторжения монгольского флота Хана Хубилая в 1281 году. Притязания французов на Флориду прекратились в 1565 году, когда ураган уничтожил французский флот, дав Испании возможность завоевать форт Каролину. Совсем недавно ураган Катрина заставил более одного миллиона человек переселиться с центрального побережья Мексиканского залива в США, создав самую крупную диаспору в истории Соединённых Штатов"
tokens = word_tokenize(text)
stemmed_words = [stemmer.stem(word) for word in tokens]
print(stemmed_words)

['погод', 'игра', 'больш', ',', 'а', 'иногд', 'даж', 'реша', 'рол', 'в', 'человеческ', 'истор', '.', 'помим', 'изменен', 'климат', ',', 'котор', 'вызыва', 'постепен', 'миграц', 'народ', '(', 'например', ',', 'опустыниван', 'ближн', 'восток', 'и', 'формирован', 'сухопутн', 'мост', 'межд', 'материк', 'во', 'врем', 'ледников', 'период', ')', ',', 'экстремальн', 'погодн', 'явлен', 'вызыва', 'меньш', 'по', 'масштаб', 'перемещен', 'народ', 'и', 'принима', 'непосредствен', 'участ', 'в', 'историческ', 'событ', '.', 'одн', 'из', 'так', 'случа', 'явля', 'спасен', 'япон', 'ветр', 'камикадз', 'от', 'вторжен', 'монгольск', 'флот', 'хан', 'хубил', 'в', '1281', 'год', '.', 'притязан', 'француз', 'на', 'флорид', 'прекрат', 'в', '1565', 'год', ',', 'когд', 'урага', 'уничтож', 'французск', 'флот', ',', 'дав', 'испан', 'возможн', 'завоева', 'форт', 'каролин', '.', 'совс', 'недавн', 'урага', 'катрин', 'застав', 'бол', 'одн', 'миллион', 'человек', 'пересел', 'с', 'центральн', 'побереж', 'мексиканск', 'зали

В силу реализации библиотек pyaspeller (каждый раз отправляет запрос) и pymystem3 (использует для обработки внешний исполняемый файл mystem.exe, из-за чего обработка замедляется) я немного изменила очередность этапов обработки и сделала батчами (кроме удаления самых часто встречающихся слов).

In [49]:
PUNCT_TO_REMOVE = '!"#$%&\'()*+,—./:;<=>?@[\\]^_`{|}~'
STOPWORDS = set(stopwords.words('russian'))


def preprocess_text(text: str):
    # Удаление пунктуации
    text = text.translate(str.maketrans('-', ' ', PUNCT_TO_REMOVE))

    # Удаление стоп-слов
    text = " ".join([word for word in text.split() if word not in STOPWORDS])

    return text

In [50]:
def contains_non_russian_or_non_digit(text):
    pattern = r'[^а-яё0-9]'
    return bool(re.search(pattern, text))

def get_pos(token):
    analysis = token.get('analysis')
    if not analysis:
        return None

    gr = analysis[0].get('gr', '')
    return gr.split('=')[0].split(',')[0]
    
    
def lemmatize(batch):
    m = Mystem()
    speller = YandexSpeller(lang='ru')

    # Объединение нескольких текстов для уменьшения частоты вызовов внешних файлов/запросов
    merged_text = ' '.join([t.lower() + ' SP ' for t in batch])

    # Исправление ошибок
    checked_text = speller.spelled(merged_text)

    # Только прилагательные и существительные 
    pos = ['A', 'S']  

    doc = set()
    res = []

    # Морфологический разбор
    tokens = m.analyze(preprocess_text(checked_text))
    nouns_and_adjectives = []

    for t in tokens:
        text = t.get('text').strip()
        
        # латинские обозначения, названия компаний и т. д.
        if contains_non_russian_or_non_digit(text):
            nouns_and_adjectives.append(text)
            continue
            

        if len(text) < 2 and text !='SP':
            continue
        elif isinstance(t.get('analysis'), list) and len(t.get('analysis'))==0:
            nouns_and_adjectives.append(text)
        elif get_pos(t) in pos:
            nouns_and_adjectives.append(t.get('analysis')[0]['lex'])
        
    # Фильтрация частей речи и разделение на отдельные документы
    for t in nouns_and_adjectives:
        if t != '\n' and t.strip() != '':
            if t != 'SP':
                doc.add(t)
            else:
                res.append(' '.join(list(doc)))
                doc = set()

    return res


In [52]:
preprocessed_contexts = contexts['context'].copy()

batch_size = 200
n = (len(preprocessed_contexts) // batch_size) * batch_size
for i in range(0, len(preprocessed_contexts)-batch_size, batch_size):
    preprocessed_contexts.iloc[i: i + batch_size] = lemmatize(preprocessed_contexts.iloc[i: i + batch_size].values)

preprocessed_contexts.iloc[n:] = lemmatize(preprocessed_contexts.iloc[n:].values)

In [53]:
from collections import Counter
cnt = Counter()
for text in preprocessed_contexts.values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(20)

[('год', 5546),
 ('время', 3051),
 ('часть', 1649),
 ('новый', 1605),
 ('большой', 1576),
 ('система', 1544),
 ('человек', 1410),
 ('век', 1390),
 ('вид', 1389),
 ('число', 1222),
 ('основной', 1213),
 ('город', 1210),
 ('работа', 1157),
 ('высокий', 1143),
 ('страна', 1085),
 ('начало', 1075),
 ('образ', 1039),
 ('результат', 1026),
 ('развитие', 1026),
 ('форма', 1025)]

In [54]:
FREQWORDS = set([w for (w, _) in cnt.most_common(20)])
def remove_freqwords(text):
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

In [55]:
preprocessed_contexts = preprocessed_contexts.apply(lambda text: remove_freqwords(text))

In [56]:
contexts['preprocessed'] = preprocessed_contexts
print(f'Средняя длина контекста после обработки: {contexts["preprocessed"].apply(len).mean()}')
print(f'Средняя длина контекста до обработки: {contexts["context"].apply(len).mean()}')

print(f'Медианная длина контекста после обработки: {contexts["preprocessed"].apply(len).median()}')
print(f'Медианная длина контекста до обработки: {contexts["context"].apply(len).median()}')

Средняя длина контекста после обработки: 344.38490621988285
Средняя длина контекста до обработки: 570.2961672473867
Медианная длина контекста после обработки: 322.0
Медианная длина контекста до обработки: 539.0


In [58]:
contexts.to_csv('preprocessed_contexts.csv', index=False)

In [59]:
questions_train = pd.read_csv('questions_train.csv', index_col=0)
questions_val = pd.read_csv('questions_val.csv', index_col=0)
questions_test = pd.read_csv('questions_test.csv', index_col=0)
questions_train.head()

,id,question,context_id
0,62310,чем представлены органические остатки?,1
1,28101,что найдено в кремнистых сланцах железорудной ...,1
2,48834,что встречается в протерозойских отложениях?,1
3,83056,что относится к числу древнейших растительных ...,1
4,5816,как образовалось графито-углистое вещество?,1


In [60]:
def preprocess_questions(df: pd.DataFrame):
    preprocessed = df['question'].apply(preprocess_text) 

    batch_size = 1000
    n = (len(preprocessed) // batch_size) * batch_size
    for i in range(0, len(preprocessed)-batch_size, batch_size):
        preprocessed.iloc[i: i + batch_size] = lemmatize(preprocessed.iloc[i: i + batch_size].values)

    preprocessed.iloc[n:] = lemmatize(preprocessed.iloc[n:].values)
    preprocessed = preprocessed.apply(lambda text: remove_freqwords(text))
    return preprocessed

In [61]:
questions_train['preprocessed'] = preprocess_questions(questions_train)
questions_val['preprocessed'] = preprocess_questions(questions_val)
questions_test['preprocessed'] = preprocess_questions(questions_test)

Видно, что длина некоторых предобработанных вопросов равна нулю, однако большинство из них 

In [62]:
questions_train[questions_train['preprocessed'].apply(len)==0]

,id,question,context_id,preprocessed
834,71879,В каком году выходит Бунтующий человек ?,149,
1522,35372,Когда это произошло?,275,
1736,25385,Благодаря чему быстро развивался город ?,311,
1737,39037,Куда был перенесен город в 500-х годах?,311,
2181,43935,Что он хотел этим предвнести?,391,
...,...,...,...,...
44161,40472,На что он нападал?,8784,
44258,59140,Сколько она продолжалась?,8807,
44937,55096,Что стало новым?,8977,
45038,74252,Чем дирижирует человек?,9000,


In [63]:
questions_val[questions_val['preprocessed'].apply(len)==0]

,id,question,context_id,preprocessed
149,26855,Чего пока ещё не появилось?,4291,
211,71521,Каковы результаты?,8866,
677,82665,Что было ими составлено?,1909,
678,47334,Что было ими было заложено?,1909,
853,17710,Сколько раз можно заменять либеро?,4164,
953,10498,Что произошло в 1862 году?,8574,
1568,39814,"Что сделали эукариоты,внедрившись в эту систему?",275,
1747,13900,куда россини уезжает гастролировать,8511,
1944,63147,Чему будет соответствовать каждая часть?,7829,
2210,32099,Как она определяется?,7193,


In [64]:
questions_test[questions_test['preprocessed'].apply(len)==0]

,id,question,context_id,preprocessed
7,47482,Что означает дифференцированы?,9079,
1045,28206,Для чего это делалось?,9252,
1851,25413,Чего можно избежать?,9393,
1889,21550,Что этому способствовало?,9399,
2215,31874,Что было открыто в 1911 году?,9453,
...,...,...,...,...
22344,61036,Что планируется начать в 2011 году?,13173,
22551,12303,Кем она разработана?,13214,
22916,83497,Как по-прусски же их называли?,13287,
22934,57803,Что низко оценивают в стране?,13290,


In [65]:
questions_train = questions_train[questions_train['preprocessed'].apply(len)>0]
questions_val = questions_val[questions_val['preprocessed'].apply(len)>0]
questions_test = questions_test[questions_test['preprocessed'].apply(len)>0]

In [66]:
os.chdir(os.path.join('..', 'preprocessed'))

questions_train.to_csv('questions_train.csv', index=False)
questions_val.to_csv('questions_val.csv', index=False)
questions_test.to_csv('questions_test.csv', index=False)

<h3>Загрузка данных в базу знаний </h3>

In [3]:
os.chdir('../../')

In [12]:
from app.database.main_database import DataRepository
from app.database.utils import create_tables

await create_tables()
dr = DataRepository()

In [13]:
await dr.add_documents(contexts)